#FactorCL on Synthetic Dataset

In [1]:
!git clone https://github.com/pliang279/FactorCL

Cloning into 'FactorCL'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 104 (delta 47), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (104/104), 268.97 KiB | 5.72 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [1]:
%cd FactorCL

/content/FactorCL


In [2]:
from Synthetic.dataset import*
from synthetic_models import*
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression

In [3]:
# Hyperparameters
A_dim, B_dim = 100, 100
x1_dim, x2_dim = 100, 100
y_dim = 1
label_dim = 1
lr = 1e-4
relative_ratio = 0.001
hidden_dim=512
embed_dim=128
layers=1
activation = 'relu'

In [4]:

# Define custom dimensions of features and labels
feature_dim_info = dict()
label_dim_info = dict()

intersections = get_intersections(num_modalities=2)

feature_dim_info['12'] = 10
feature_dim_info['1'] = 6
feature_dim_info['2'] = 6

label_dim_info['12'] = 10
label_dim_info['1'] = 6
label_dim_info['2'] = 6

print(intersections)
print(feature_dim_info)
print(label_dim_info)

# Get datasets
total_data, total_labels, total_raw_features = generate_data(30000, 2, feature_dim_info, label_dim_info)
total_labels = get_labels(label_dim_info, total_raw_features)

dataset = MultimodalDataset(total_data, total_labels)

# Dataloader
batch_size = 256
num_data = total_labels.shape[0]
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8*num_data), num_data-int(0.8*num_data)])

train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                            batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)
data_loader = DataLoader(dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)


['1', '2', '12']
{'12': 10, '1': 6, '2': 6}
{'12': 10, '1': 6, '2': 6}


##FactorCL-SUP

In [5]:
factorcl_sup = FactorCLSUP(A_dim, B_dim, 20, hidden_dim, embed_dim).cuda()
train_sup_model(factorcl_sup, train_loader, dataset, num_epoch=10, num_club_iter=1)
factorcl_sup.eval()

iter:  0  i_batch:  0  loss:  9.623612277209759e-05
iter:  1  i_batch:  0  loss:  -3.0933303833007812
iter:  2  i_batch:  0  loss:  -5.884997367858887
iter:  3  i_batch:  0  loss:  -7.007236480712891
iter:  4  i_batch:  0  loss:  -7.123583793640137
iter:  5  i_batch:  0  loss:  -7.228031158447266
iter:  6  i_batch:  0  loss:  -7.025860786437988
iter:  7  i_batch:  0  loss:  -7.642902374267578
iter:  8  i_batch:  0  loss:  -8.002883911132812
iter:  9  i_batch:  0  loss:  -7.967354774475098


FactorCLSUP(
  (backbones): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
  )
  (linears_infonce_x1x2): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (linears_club_x1x2_cond): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (li

In [7]:
# Simple evaluation using linear logistic regression model

# Embeddings
train_embeds = factorcl_sup.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = factorcl_sup.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
score

0.9863333333333333

##FactorCL-SSL

In [7]:
factorcl_ssl = FactorCLSSL(A_dim, B_dim, hidden_dim, embed_dim).cuda()
train_sup_model(factorcl_ssl, train_loader, dataset, num_epoch=10, num_club_iter=1)
factorcl_ssl.eval()

iter:  0  i_batch:  0  loss:  3.082677721977234e-06
iter:  1  i_batch:  0  loss:  -7.57092809677124
iter:  2  i_batch:  0  loss:  -8.953313827514648
iter:  3  i_batch:  0  loss:  -10.78848648071289
iter:  4  i_batch:  0  loss:  -12.652485847473145
iter:  5  i_batch:  0  loss:  -14.410904884338379
iter:  6  i_batch:  0  loss:  -14.283191680908203
iter:  7  i_batch:  0  loss:  -14.439704895019531
iter:  8  i_batch:  0  loss:  -15.733597755432129
iter:  9  i_batch:  0  loss:  -15.78852653503418


FactorCLSSL(
  (backbones): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
  )
  (linears_infonce_x1x2): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=T

In [8]:
# Simple evaluation using linear logistic regression model

# Embeddings
train_embeds = factorcl_ssl.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = factorcl_ssl.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
score

0.9716666666666667

##SimCLR

In [13]:
# set use_label=True for SupCon model
simclr_model = SupConModel(A_dim, B_dim, hidden_dim, embed_dim, use_label=False).cuda()
simclr_optim = optim.Adam(simclr_model.parameters(), lr=lr)
train_supcon(simclr_model, train_loader, simclr_optim, num_epoch=20)
simclr_model.eval()

iter:  0  i_batch:  0  loss:  11.985010147094727
iter:  1  i_batch:  0  loss:  5.873631477355957
iter:  2  i_batch:  0  loss:  4.937842845916748
iter:  3  i_batch:  0  loss:  4.577206611633301
iter:  4  i_batch:  0  loss:  3.8152596950531006
iter:  5  i_batch:  0  loss:  2.4593682289123535
iter:  6  i_batch:  0  loss:  1.589794635772705
iter:  7  i_batch:  0  loss:  1.177107810974121
iter:  8  i_batch:  0  loss:  0.9892462491989136
iter:  9  i_batch:  0  loss:  0.9151424169540405
iter:  10  i_batch:  0  loss:  0.809130072593689
iter:  11  i_batch:  0  loss:  0.6216927766799927
iter:  12  i_batch:  0  loss:  0.600822925567627
iter:  13  i_batch:  0  loss:  0.5172191858291626
iter:  14  i_batch:  0  loss:  0.5168259143829346
iter:  15  i_batch:  0  loss:  0.43819260597229004
iter:  16  i_batch:  0  loss:  0.5183995962142944
iter:  17  i_batch:  0  loss:  0.32648688554763794
iter:  18  i_batch:  0  loss:  0.42932015657424927
iter:  19  i_batch:  0  loss:  0.5603318810462952


SupConModel(
  (encoder_x1): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (encoder_x2): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (projection_x1): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
  )
  (projection_x2): Sequential(
    (0): Linear(in_features=128, out_features=1

In [14]:
train_embeds = simclr_model.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

# Embeddings
test_embeds = simclr_model.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
score

0.9555